## Convert KITTI Point Cloud to BEV image

In [ ]:
from pathlib import Path
from easydict import EasyDict
from tqdm import tqdm
import pickle
import yaml
import cv2

from pcdet.utils.bev_utils import get_sample, pointcloud3d_to_bevimage2d, show_bev_image_preview
from pcdet.utils.bev_utils import boxes3d_lidar_to_rotated_bev_boxes, draw_bev_boxes, save_bev_images_and_boxes


dataset_cfg = EasyDict(yaml.safe_load(open('/home/rlab10/OpenPCDet/lidar2bev/master_thesis/dataset_configs/kitti_dataset_bev_custom.yaml')))
class_names = ['Car', 'Pedestrian', 'Cyclist']
file_path = '/home/rlab10/OpenPCDet/lidar2bev/master_thesis/lidar3d_to_bev2d.ipynb' 
NOTEBOOK_DIR = (Path(file_path).resolve().parent).resolve()
ROOT_DIR = Path('/home/rlab10/OpenPCDet')
DATA_VARIANT = dataset_cfg.DATA_VARIANT
MODE = 'train' if dataset_cfg.DATA_SPLIT['train'] else 'val'

data_path_kitti = ROOT_DIR / 'data' / 'kitti' # raw data path
save_path_kitti = NOTEBOOK_DIR / 'kitti'

box_colormap = [
    [1, 1, 1], # not assigned
    [0, 1, 0], # Car (green)
    [1, 0, 1], # Pedestrian (violet)
    [1, 1, 0], # Cyclist (yellow)
]

pkl_paths = {
    'train': {
        'default': data_path_kitti / dataset_cfg.PKL_TRAIN_DEFAULT,
        'densified': data_path_kitti / dataset_cfg.PKL_TRAIN_DENSIFIED,
        'upsampled': data_path_kitti / dataset_cfg.PKL_TRAIN_UPSAMPLED,
        'interpolated': data_path_kitti / dataset_cfg.PKL_TRAIN_INTERPOLATED,
    },
    'val': {
        'default': data_path_kitti / dataset_cfg.PKL_VAL_DEFAULT,
        'densified': data_path_kitti / dataset_cfg.PKL_VAL_DENSIFIED,
        'upsampled': data_path_kitti / dataset_cfg.PKL_VAL_UPSAMPLED,
        'interpolated': data_path_kitti / dataset_cfg.PKL_VAL_INTERPOLATED,
    }
}

pkl_path = pkl_paths[MODE][DATA_VARIANT]
compression_params = [cv2.IMWRITE_PNG_COMPRESSION, 0] # no compression



def main():
    print(f"Mode: {MODE}, Variant: {DATA_VARIANT}")
    print(f"Load: {pkl_path}")
    with open(pkl_path, 'rb') as f:
        data_list = pickle.load(f)

    num_samples = len(data_list)
    # Original sample + 6 Augmentations else no augmentations
    num_augments = 7 if MODE == 'train' else 1 
    print(f"Samples: {num_samples}")
    print(f"Augmentations: {num_augments}")
    print(f"All: {num_samples * num_augments}")

    # DEBUG - single samples
    data_list = [data_list[4]]

    # main loop
    for sample_idx in tqdm(range(len(data_list)), desc="Samples"):
        for augment_idx in tqdm(range(num_augments), disable=True):
            sample = get_sample(data=data_list, sample_idx=sample_idx, augment_idx=augment_idx, mode=MODE)
            bev_image, _ = pointcloud3d_to_bevimage2d(points=sample['points'], cfg=dataset_cfg, num_slices=dataset_cfg.NUM_SLICES, 
                                                      filter_points=False)
            valid_bev_boxes = []
            if 'gt_boxes' in sample:
                for boxes3d in sample['gt_boxes']:
                    bev_box = boxes3d_lidar_to_rotated_bev_boxes(
                        bev_img=bev_image,
                        bev_image_height=dataset_cfg.BEV_IMAGE_HEIGHT,
                        bev_image_width=dataset_cfg.BEV_IMAGE_WIDTH,
                        bev_res=dataset_cfg.CELL_SIZE,
                        boxes3d=boxes3d,
                        min_points=5
                    )
                    if bev_box is not None:
                        cls_idx = int(boxes3d[7])
                        draw_bev_boxes(bev_image=bev_image, bev_box=bev_box, cls_idx=cls_idx, box_colormap=box_colormap, 
                                       thickness=1, obj_direction_color=(0,255,255))

                        valid_bev_boxes.append(cls_idx, bev_box)

            key = show_bev_image_preview(bev_image, BGR2RGB=True, win_size=(800, 800))

            if valid_bev_boxes and dataset_cfg.SAVE_FLAG:
                save_bev_images_and_boxes(save_path=save_path_kitti, frame_id=sample['frame_id'], bev_image_bgr=bev_image, 
                                          valid_bev_boxes=valid_bev_boxes, augment_idx=augment_idx, compression=compression_params,
                                          normalize_coords=dataset_cfg.NORM_BOX_COORDS, data_variant=DATA_VARIANT)

            if key == ord('q'):
                break
        if key == ord('q'):
            break
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()

## Convert ZOD Point Cloud to BEV image

In [ ]:
from pathlib import Path
from easydict import EasyDict
from tqdm import tqdm
import pickle
import yaml
import cv2

from pcdet.utils.bev_utils import get_sample, pointcloud3d_to_bevimage2d, show_bev_image_preview
from pcdet.utils.bev_utils import boxes3d_lidar_to_rotated_bev_boxes, draw_bev_boxes, save_bev_images_and_boxes


dataset_cfg = EasyDict(yaml.safe_load(open('/home/rlab10/OpenPCDet/lidar2bev/master_thesis/dataset_configs/zod_dataset_bev_custom.yaml')))
class_names = ['Car', 'Pedestrian', 'Cyclist']
file_path = '/home/rlab10/OpenPCDet/lidar2bev/master_thesis/lidar3d_to_bev2d.ipynb' 
NOTEBOOK_DIR = (Path(file_path).resolve().parent).resolve()
ROOT_DIR = Path('/home/rlab10/OpenPCDet')
DATA_VARIANT = dataset_cfg.DATA_VARIANT
MODE = 'train' if dataset_cfg.DATA_SPLIT['train'] else 'val'

data_path_zod = ROOT_DIR / 'data' / 'zod' # raw data path
save_path_zod = NOTEBOOK_DIR / 'zod'

box_colormap = [
    [1, 1, 1], # not assigned
    [0, 1, 0], # Car (green)
    [1, 0, 1], # Pedestrian (violet)
    [1, 1, 0], # Cyclist (yellow)
]

pkl_paths = {
    'train': {
        'default': data_path_zod / dataset_cfg.PKL_TRAIN_DEFAULT,
    },
    'val': {
        'default': data_path_zod / dataset_cfg.PKL_VAL_DEFAULT,
    }
}

pkl_path = pkl_paths[MODE][DATA_VARIANT]
compression_params = [cv2.IMWRITE_PNG_COMPRESSION, 0] # no compression



def main():
    print(f"Mode: {MODE}, Variant: {DATA_VARIANT}")
    print(f"Load: {pkl_path}")
    with open(pkl_path, 'rb') as f:
        data_list = pickle.load(f)

    num_samples = len(data_list)
    # Original sample + 6 Augmentations else no augmentations
    num_augments = 7 if MODE == 'train' else 1 
    print(f"Samples: {num_samples}")
    print(f"Augmentations: {num_augments}")
    print(f"All: {num_samples * num_augments}")

    # DEBUG - single samples
    data_list = [data_list[4]]

    # main loop
    for sample_idx in tqdm(range(len(data_list)), desc="Samples"):
        for augment_idx in tqdm(range(num_augments), disable=True):
            sample = get_sample(data=data_list, sample_idx=sample_idx, augment_idx=augment_idx, mode=MODE)
            bev_image, _ = pointcloud3d_to_bevimage2d(points=sample['points'], cfg=dataset_cfg, num_slices=dataset_cfg.NUM_SLICES, 
                                                      filter_points=False)
            valid_bev_boxes = []
            if 'gt_boxes' in sample:
                for boxes3d in sample['gt_boxes']:
                    bev_box = boxes3d_lidar_to_rotated_bev_boxes(
                        bev_img=bev_image,
                        bev_image_height=dataset_cfg.BEV_IMAGE_HEIGHT,
                        bev_image_width=dataset_cfg.BEV_IMAGE_WIDTH,
                        bev_res=dataset_cfg.CELL_SIZE,
                        boxes3d=boxes3d,
                        min_points=5
                    )
                    if bev_box is not None:
                        cls_idx = int(boxes3d[7])
                        draw_bev_boxes(bev_image=bev_image, bev_box=bev_box, cls_idx=cls_idx, box_colormap=box_colormap, 
                                       thickness=1, obj_direction_color=(0,255,255))

                        valid_bev_boxes.append(bev_box)

            key = show_bev_image_preview(bev_image, BGR2RGB=True, win_size=(800, 800))

            if valid_bev_boxes and dataset_cfg.SAVE_FLAG:
                save_bev_images_and_boxes(save_path=save_path_kitti, frame_id=sample['frame_id'], bev_image_bgr=bev_image, 
                                          valid_bev_boxes=valid_bev_boxes, augment_idx=augment_idx, compression=compression_params,
                                          normalize_coords=dataset_cfg.NORM_BOX_COORDS, data_variant=DATA_VARIANT)

            if key == ord('q'):
                break
        if key == ord('q'):
            break
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()